<a href="https://colab.research.google.com/github/emolinaperez/econometrics_mek/blob/main/Week%202/R/Week2_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Econometrics

load library for importing database into R (from stata format)


In [ ]:
library(foreign)
library(dplyr)
library(ggplot2)

load data

In [ ]:
jobcorpsfile<-"https://raw.githubusercontent.com/emolinaperez/econometrics_mek/main/Week%202/data/jobcorps.dta"
JobCorps<-read.dta(jobcorpsfile, convert.dates = TRUE, convert.factors = TRUE,missing.type = FALSE,convert.underscore = FALSE, warn.missing.labels = TRUE)


explore file

In [ ]:
head(JobCorps)

In [ ]:
dim(JobCorps)

In [ ]:
attributes(JobCorps)

In [ ]:
str(JobCorps)

In [ ]:
summary(JobCorps)

Explore in class characteristics of differnet variables

Compare hispanics in control and treatment group

First compare means

In control group

In [ ]:
mean(JobCorps[JobCorps$treatmnt==0,"hispanic"] ,na.rm=TRUE)

In [ ]:
sum(JobCorps[JobCorps$treatmnt==0,"hispanic"] ,na.rm=TRUE)

In treatment group

In [ ]:
mean(JobCorps[JobCorps$treatmnt==1,"hispanic"],na.rm=TRUE)

In [ ]:
sum(JobCorps[JobCorps$treatmnt==1,"hispanic"] ,na.rm=TRUE)

In [ ]:
# Calculate means and confidence intervals for "hispanic" by treatment group
treatment_means <- JobCorps %>%
  group_by(treatmnt) %>%
  summarize(
    mean_hispanic = mean(hispanic, na.rm = TRUE),
    ci_lower = mean(hispanic, na.rm = TRUE) - qt(0.975, df = n() - 1) * sd(hispanic, na.rm = TRUE) / sqrt(n()),
    ci_upper = mean(hispanic, na.rm = TRUE) + qt(0.975, df = n() - 1) * sd(hispanic, na.rm = TRUE) / sqrt(n())
  )
treatment_means

In [ ]:
p1<- aggregate(list(mean_hispanic=JobCorps$hispanic),list(treatment_type=JobCorps$treatmnt),mean,na.rm=TRUE)
p2 <- aggregate(list(ci_lower_hispanic=JobCorps$hispanic),list(treatment_type=JobCorps$treatmnt), function(x) { conf_level<-0.95; mean_x <- mean(x,na.rm=TRUE);
                                                                                                      se_x <- sd(x,na.rm=TRUE) / sqrt(length(x));
                                                                                                      df <- length(x) - 1;
                                                                                                      t_value <- qt((1 + conf_level) / 2, df);
                                                                                                      ci_lower <- mean_x - t_value * se_x;
                                                                                                      ci_lower } )

p3 <- aggregate(list(ci_higher_hispanic=JobCorps$hispanic),list(treatment_type=JobCorps$treatmnt), function(x) { conf_level<-0.95; mean_x <- mean(x,na.rm=TRUE);
                                                                                                      se_x <- sd(x,na.rm=TRUE) / sqrt(length(x));
                                                                                                      df <- length(x) - 1;
                                                                                                      t_value <- qt((1 + conf_level) / 2, df);
                                                                                                      ci_higher <- mean_x + t_value * se_x;
                                                                                                      ci_higher } )
Reduce(function(...) merge(...,by="treatment_type"), list(p1,p2,p3))

In [ ]:
x<- JobCorps$hispanic[JobCorps$treatmnt==1]
sd(x)
#conf_level<-0.95
#mean_x <- mean(x,na.rm=TRUE);
 #                                                                                                     se_x <- sd(x) / sqrt(length(x))
  #                                                                                                    df <- length(x) - 1
   #                                                                                                   t_value <- qt((1 + conf_level) / 2, df)
    #                                                                                                  ci_lower <- mean_x - t_value * se_x
     #                                                                                                 ci_lower

In [ ]:
# Bar plot with confidence intervals
ggplot(treatment_means, aes(x = factor(treatmnt), y = mean_hispanic, fill = factor(treatmnt))) +
  geom_bar(stat = "identity", alpha = 0.6) +                          # Bar for mean
  geom_errorbar(aes(ymin = ci_lower, ymax = ci_upper), width = 0.2,   # Confidence intervals
                color = "black") +
  labs(
    title = "Mean Proportion of Hispanics by Treatment",
    x = "Treatment (0 = Control, 1 = Treated)",
    y = "Mean Proportion of Hispanics",
    fill = "Treatment"
  ) +
  theme_minimal()

In [ ]:
install.packages('Hmisc')
library(Hmisc)

ggplot(JobCorps, aes(x = factor(treatmnt), y = hispanic, fill = factor(treatmnt))) +
  stat_summary(fun = mean, geom = "bar", alpha = 0.6) +
  stat_summary(
    fun.data = mean_cl_normal,                                         # 95% confidence intervals
    geom = "errorbar",
    width = 0.2
  ) +
  labs(
    title = "Mean Proportion of Hispanics by Treatment",
    x = "Treatment (0 = Control, 1 = Treated)",
    y = "Mean Proportion of Hispanics",
    fill = "Treatment"
  ) +
  scale_fill_brewer(palette = "Set2") +
  theme_minimal()

Difference in means test

In [ ]:
mytest <- with( JobCorps, t.test(hispanic~treatmnt,alternative = c("two.sided"),conf.level = 0.95))
mytest

In [ ]:
t.test(JobCorps$hispanic~JobCorps$treatmnt,alternative = c("two.sided"),conf.level = 0.95)

In [ ]:
t.test(JobCorps$male~JobCorps$treatmnt,alternative = c("two.sided"),conf.level = 0.95)

conclusion is not possible to reject null hypothesis that hispanic in both groups are the same


use OLS model to see of randomization was sucessful

 first define vector of variables of interest

In [ ]:
target.vars<-colnames(JobCorps)[19:36]
target.vars

In [ ]:
target.vars<-subset(colnames(JobCorps), grepl("earnq",colnames(JobCorps))==FALSE)
target.vars

In [ ]:
summary(JobCorps[, target.vars])

In [ ]:
cor_matrix <- cor(JobCorps[, c("white", "black", "hispanic", "treatmnt")], use = "complete.obs")
cor_matrix

In [ ]:
heatmap(cor_matrix, symm = TRUE, col = colorRampPalette(c("blue", "white", "red"))(50),
        main = "Heatmap of Correlations")

run the regression, first a simple example with 3 observable caracteristics


In [ ]:
model1<-lm(treatmnt~ white+black+hispanic, data=JobCorps)
summary(model1)

In [ ]:
plot(model1)

 now all variables

  first save the model

In [ ]:
formula.model2<-as.formula(paste("treatmnt","~",paste(target.vars,collapse="+"),sep=""))
formula.model2


second run regression

In [ ]:
model2<-lm(formula.model2,data=JobCorps)
summary(model2)

selection on observables

In [ ]:
formula.model3<-as.formula(paste("attrit","~",paste(target.vars,collapse="+"),sep=""))
formula.model3

second run regression

In [ ]:
model3<-lm(attrit~ white+black+hispanic, data=JobCorps)
summary(model3)

In [ ]:
model3<-lm(formula.model3, data=JobCorps)
summary(model3)

In [ ]:
test<-subset(JobCorps,complete.cases(JobCorps[,target.vars])==TRUE)

In [ ]:
dim(test)

In [ ]:
cor(test[,target.vars])

Average Treatment Effect  

In [ ]:
formula.model_ATE<-as.formula(earnq16~treatmnt)
formula.model_ATE

In [ ]:
model_ATE<-lm(formula.model_ATE,data=JobCorps, na.action = na.omit)
summary(model_ATE)

Is this a really ATE ?

Effect on the Treated

In [ ]:
summary(subset(JobCorps,treatmnt==1)$earnq16)

In [ ]:
ToT<-mean(subset(JobCorps,treatmnt==1)$earnq16,na.rm=TRUE)  # ToT: Average Effect of Treatment on the Treated
ToT

In [ ]:
NT<-mean(subset(JobCorps,treatmnt==0)$earnq16,na.rm=TRUE)  # ToT: Average Effect of Treatment on the Treated
NT

ATE , maybe

In [ ]:
ToT-NT

compare this to regression coefficient

In [ ]:
 model_ATE$coefficients

Randomization seems to be working

In [ ]:
summary(model_ATE$residuals) #what is the expected value?

is this correlated with the treatment

In [ ]:
cor(model_ATE$residuals,subset(JobCorps,complete.cases(JobCorps[,c("earnq16","treatmnt")])==TRUE)$treatmnt)

no correlation at all, what does this mean?